## Modelos seq2seq + atención



Los modelos de atención han transformado la forma en que abordamos las tareas de traducción automática y otras aplicaciones de procesamiento de lenguaje natural (PLN). Estos modelos permiten que el decodificador se centre en diferentes partes de la secuencia de entrada mientras genera cada palabra de la secuencia de salida. Dos mecanismos importantes en este contexto son la atención global y la atención local. Este informe detalla estos mecanismos, sus implementaciones, y sus aplicaciones prácticas.

#### Mecanismo de atención global

**Descripción general**

El mecanismo de atención global, introducido por Bahdanau también conocido como atención suave, permite que el decodificador considere todas las posiciones de la secuencia de entrada al generar cada palabra de la secuencia de salida. Este enfoque asegura que el modelo tenga acceso a toda la información de la entrada en cada paso del proceso de decodificación, mejorando la calidad de la traducción, especialmente en secuencias largas y complejas.

**Ecuaciones y cálculos**

El mecanismo de atención global se basa en los siguientes pasos y ecuaciones:

1. **Cálculo de los puntajes de atención**:
   Para cada paso de tiempo del decodificador, se calcula un puntaje de atención $ e_{ij}$ que mide la afinidad entre el estado oculto del decodificador en el paso de tiempo $ j$, denotado como $ s_{j-1}$, y el estado oculto del codificador en el paso de tiempo $ i$, denotado como $ h_i$. Esto se puede calcular usando una red neuronal feedforward con una sola capa oculta (o cualquier otra función de afinidad):

   $$
   e_{ij} = v^T \tanh(W_1 h_i + W_2 s_{j-1})
   $$

   donde $W_1$ y $W_2$ son matrices de peso aprendibles y $v$ es un vector de peso aprendible.

2. **Normalización de puntajes de atención**:
   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención $ \alpha_{ij}$, que son distribuciones de probabilidad sobre las posiciones de la secuencia de entrada:

 $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})}
  $$

3. **Cálculo del vector de contexto**:
   El vector de contexto $ c_j$ para cada paso de tiempo del decodificador se calcula como una combinación ponderada de los estados ocultos del codificador:

 $$
   c_j = \sum_{i=1}^{T_x} \alpha_{ij} h_i
 $$

4. **Generación de la salida del decodificador**:
   Finalmente, el vector de contexto $ c_j$ se combina con el estado oculto del decodificador $ s_j$ para generar la salida $ y_j$:

 $$
   y_j = g(c_j, s_j)
 $$

  donde $ g$ puede ser una función no lineal como una red neuronal.
    


La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención global dentro de un modelo seq2seq:

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(1)
        H = hidden.repeat(max_len, 1, 1).transpose(0, 1)
        energy = torch.tanh(self.attn(torch.cat([H, encoder_outputs], 2)))
        energy = energy.transpose(1, 2)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)
        energy = torch.bmm(v, energy)
        attn_weights = F.softmax(energy.squeeze(1), dim=1)
        return attn_weights

class Seq2SeqWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2SeqWithAttention, self).__init__()
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.GRU(output_dim, hidden_dim, batch_first=True)
        self.attention = Attention(hidden_dim)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, src, trg):
        encoder_outputs, hidden = self.encoder(src)
        decoder_outputs, hidden = self.decoder(trg, hidden)
        attn_weights = self.attention(hidden, encoder_outputs)
        context = attn_weights.unsqueeze(1).bmm(encoder_outputs)
        output = torch.cat([hidden.squeeze(0), context.squeeze(1)], 1)
        output = self.fc(output)
        return output


#### Mecanismo de atención local

**Descripción general**

El mecanismo de atención local, propuesto por Luong et al. (2015), reduce la complejidad computacional al limitar el alcance de la atención a una ventana local alrededor de cada posición de la secuencia de entrada. Este enfoque es particularmente útil en secuencias largas, donde la atención global puede ser computacionalmente costosa.

**Ecuaciones y cálculos**

El mecanismo de atención local se define a través de los siguientes pasos:

1. **Predicción de la posición de atención**:
   Primero, se predice una posición de atención $p_j$ para cada paso de tiempo $j$ del decodificador. Esto puede hacerse mediante una simple función lineal o una red neuronal:
   

   $$
   p_j = S \cdot \sigma(W_p s_{j-1})
   $$ 

   donde $S$ es la longitud de la secuencia de entrada, $\sigma$ es la función sigmoide, $W_p$ es una matriz de peso aprendible, y $s_{j-1}$ es el estado oculto del decodificador en el paso $j-1$.


2. **Definición de la ventana local**:
   Se define una ventana local de tamaño $2D + 1$ centrada en $p_j$. Los límites de la ventana se calculan como:

   $$
   [p_j - D, p_j + D]
  $$ 


3. **Cálculo de puntajes de atención dentro de la ventana**:
   Los puntajes de atención $e_{ij}$ se calculan solo para las posiciones dentro de la ventana local:


   $$
   e_{ij} = v^T \tanh(W_1 h_i + W_2 s_{j-1})
  $$ 


4. **Normalización de puntajes de atención**:
   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención $\alpha_{ij}$:

   $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k \in [p_j - D, p_j + D]} \exp(e_{ik})}
  $$ 

5. **Cálculo del vector de contexto**:
   El vector de contexto $c_j$ se calcula como una combinación ponderada de los estados ocultos del codificador dentro de la ventana local:

   $$
   c_j = \sum_{i \in [p_j - D, p_j + D]} \alpha_{ij} h_i
  $$ 



La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención local dentro de un modelo seq2seq:

In [ ]:
class LocalAttention(nn.Module):
    def __init__(self, hidden_size, window_size):
        super(LocalAttention, self).__init__()
        self.window_size = window_size
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.size(0)
        max_len = encoder_outputs.size(1)
        hidden = hidden.squeeze(0)
        pos = torch.arange(max_len).unsqueeze(0).repeat(batch_size, 1)
        hidden = hidden.unsqueeze(1).repeat(1, max_len, 1)
        attn_energies = torch.tanh(self.attn(torch.cat([hidden, encoder_outputs], 2)))
        attn_energies = torch.sum(self.v * attn_energies, dim=2)
        attn_weights = F.softmax(attn_energies, dim=1)

        return attn_weights

class Seq2SeqWithLocalAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, window_size):
        super(Seq2SeqWithLocalAttention, self).__init__()
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.GRU(output_dim, hidden_dim, batch_first=True)
        self.local_attention = LocalAttention(hidden_dim, window_size)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, src, trg):
        encoder_outputs, hidden = self.encoder(src)
        decoder_outputs, hidden = self.decoder(trg, hidden)
        attn_weights = self.local_attention(hidden, encoder_outputs)
        context = attn_weights.unsqueeze(1).bmm(encoder_outputs)
        output = torch.cat([hidden.squeeze(0), context.squeeze(1)], 1)
        output = self.fc(output)
        return output


Los mecanismos de atención global y local ofrecen diferentes enfoques para mejorar la generación de secuencias en modelos seq2seq. La atención global proporciona una visión completa de la secuencia de entrada, capturando dependencias a largo plazo, mientras que la atención local mejora la eficiencia computacional al enfocarse en ventanas locales. La elección entre estos mecanismos depende de la naturaleza de la tarea y las prioridades del modelo en términos de precisión y eficiencia.


### Ejemplo 

Se implementa un modelo de red neuronal con mecanismo de atención para procesar y traducir secuencias de texto, típicamente utilizado en tareas de traducción automática, como del alemán al inglés. Este tipo de modelo pertenece a la categoría de modelos secuencia a secuencia (seq2seq) con atención, donde la "atención" permite al modelo enfocarse en diferentes partes de la entrada para cada paso de la salida.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Importar las librerías necesarias
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Definición de símbolos especiales para el proceso de codificación/decodificación
# S: Símbolo que indica el inicio de la decodificación de la entrada
# E: Símbolo que indica el inicio de la salida de la decodificación
# P: Símbolo que se utiliza para rellenar la secuencia si el tamaño del lote actual es menor que los pasos de tiempo

def make_batch():
    # Creación de lotes de datos usando one-hot encoding para las palabras en las frases
    input_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[0].split()]]]
    output_batch = [np.eye(n_class)[[word_dict[n] for n in sentences[1].split()]]]
    target_batch = [[word_dict[n] for n in sentences[2].split()]]

    # Convertir los lotes a tensores de PyTorch
    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        # Celdas RNN para el codificador y decodificador con dropout
        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)

        # Capas lineales para calcular la atención y la salida
        self.attn = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden * 2, n_class)

    def forward(self, enc_inputs, hidden, dec_inputs):
        # Transposición de las entradas para obtener la dimensión correcta
        enc_inputs = enc_inputs.transpose(0, 1)
        dec_inputs = dec_inputs.transpose(0, 1)

        # Obtención de las salidas y estados ocultos del codificador
        enc_outputs, enc_hidden = self.enc_cell(enc_inputs, hidden)

        trained_attn = []
        hidden = enc_hidden
        n_step = len(dec_inputs)
        model = torch.empty([n_step, 1, n_class])

        for i in range(n_step):  # Proceso para cada paso de tiempo
            dec_output, hidden = self.dec_cell(dec_inputs[i].unsqueeze(0), hidden)
            attn_weights = self.get_att_weight(dec_output, enc_outputs)
            trained_attn.append(attn_weights.squeeze().data.numpy())

            # Producto de matrices para obtener el contexto y combinarlo con la salida del decodificador
            context = attn_weights.bmm(enc_outputs.transpose(0, 1))
            dec_output = dec_output.squeeze(0)
            context = context.squeeze(1)
            model[i] = self.out(torch.cat((dec_output, context), 1))

        return model.transpose(0, 1).squeeze(0), trained_attn

    def get_att_weight(self, dec_output, enc_outputs):
        n_step = len(enc_outputs)
        attn_scores = torch.zeros(n_step)

        for i in range(n_step):
            attn_scores[i] = self.get_att_score(dec_output, enc_outputs[i])

        # Normalizar los puntajes para obtener pesos en el rango de 0 a 1
        return F.softmax(attn_scores, dim=0).view(1, 1, -1)

    def get_att_score(self, dec_output, enc_output):
        score = self.attn(enc_output)
        return torch.dot(dec_output.view(-1), score.view(-1))  # Producto punto para obtener un valor escalar

if __name__ == '__main__':
    n_step = 5  # Número de celdas (número de pasos)
    n_hidden = 128  # Número de unidades ocultas en una celda

    # Frases de ejemplo para la traducción
    sentences = ['ich mochte ein bier P', 'S i want a beer', 'i want a beer E']

    # Crear listas de palabras únicas y diccionarios para el mapeo de palabras a índices
    word_list = " ".join(sentences).split()
    word_list = list(set(word_list))
    word_dict = {w: i for i, w in enumerate(word_list)}
    number_dict = {i: w for i, w in enumerate(word_list)}
    n_class = len(word_dict)  # Tamaño del vocabulario

    hidden = torch.zeros(1, 1, n_hidden)  # Estado oculto inicial

    model = Attention()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    input_batch, output_batch, target_batch = make_batch()

    # Proceso de entrenamiento
    for epoch in range(2000):
        optimizer.zero_grad()
        output, _ = model(input_batch, hidden, output_batch)

        loss = criterion(output, target_batch.squeeze(0))
        if (epoch + 1) % 400 == 0:
            print('Epoca:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

    # Prueba del modelo con nuevas entradas
    test_batch = [np.eye(n_class)[[word_dict[n] for n in 'SPPPP']]]
    test_batch = torch.FloatTensor(test_batch)
    predict, trained_attn = model(input_batch, hidden, test_batch)
    predict = predict.data.max(1, keepdim=True)[1]
    print(sentences[0], '->', [number_dict[n.item()] for n in predict.squeeze()])

    # Mostrar la matriz de atención
    # Suponiendo que 'trained_attn' es tu matriz de atención y quieres visualizarla
    fig, ax = plt.subplots(figsize=(5, 5))
    cax = ax.matshow(trained_attn, cmap='viridis')

    # Supongamos que 'sentences' son tus oraciones de entrada y salida
    input_sentence = sentences[0].split()  # Oración de entrada
    output_sentence = sentences[2].split()  # Oración de salida

    # Configurando los ticks del eje X
    ax.set_xticks(range(len(input_sentence)))
    ax.set_xticklabels(input_sentence)
    ax.xaxis.set_major_locator(ticker.FixedLocator(range(len(input_sentence))))

    # Configurando los ticks del eje Y
    ax.set_yticks(range(len(output_sentence)))
    ax.set_yticklabels(output_sentence)
    ax.yaxis.set_major_locator(ticker.FixedLocator(range(len(output_sentence))))

    # Rotar las etiquetas del eje X para mejorar la visibilidad
    plt.xticks(rotation=90)

    # Añadiendo etiquetas y título si es necesario
    ax.set_xlabel('Oracion entrada ')
    ax.set_ylabel('Oracion de salida')
    ax.set_title('Matriz de atencion')

    plt.show()


La matriz de atención visualiza cómo el modelo de traducción con mecanismo de atención distribuye su "foco" o "atención" mientras traduce una secuencia de entrada a una secuencia de salida. En este caso específico, parece que la traducción va del alemán al inglés, de la frase "ich mochte ein bier P" a "i want a beer E". Vamos a analizar la matriz:


* Ejes: El eje horizontal representa las palabras de la secuencia de entrada (en alemán), mientras que el eje vertical representa las palabras de la secuencia de salida (en inglés).
* Colores: Las áreas más claras (amarillas) muestran mayor atención, es decir, cuando el modelo se centra más en esa palabra específica de la entrada al generar la palabra correspondiente de la salida. Las áreas más oscuras indican menos atención.

**Análisis detallado**

Palabra por palabra:

- "i" (output) presta atención principalmente a "ich" (input), lo cual tiene sentido ya que ambos son pronombres personales.
- "want" (output) presta atención a "mochte" (input), reflejando la relación entre el deseo expresado en ambas lenguas.
- "a" (output) y "beer" (output) ambos enfocan principalmente en "bier" (input), lo que indica que el modelo reconoce correctamente que "bier" es el objeto principal de la oración.
- "beer" (output) también muestra atención a "ein" (input), lo que es lógico dado que "ein" es el artículo indefinido para "bier".

La palabra "E" en la salida, que probablemente sea un símbolo especial para indicar el final de la frase, presta atención a la palabra "P" en la entrada. Esto podría sugerir que "P" se usa también como un marcador de posición o un símbolo especial en el modelo (posiblemente para marcar el fin de una frase o rellenar espacio si la entrada es más corta que el número máximo de pasos de tiempo esperados por el modelo).

La matriz sugiere que el mecanismo de atención está funcionando eficazmente, al concentrarse en las partes relevantes de la entrada para cada palabra de la salida. Esto es crucial para la calidad de la traducción, especialmente en oraciones más largas y complejas donde la correspondencia directa entre las palabras de entrada y salida no es tan lineal.

Observar la atención pagada a los símbolos especiales como "E" y "P" puede ofrecer conocimiento sobre cómo el modelo maneja los finales de frases y el relleno, aspectos que podrían ajustarse durante la optimización del modelo para mejorar su rendimiento o hacerlo más adaptativo a diferentes longitudes de entrada.
Esta visualización de la matriz de atención no solo ayuda a verificar la correcta funcionalidad del modelo, sino que también proporciona una herramienta valiosa para ajustar y mejorar el modelo, asegurando que la atención se distribuye de manera que refleje las estructuras lingüísticas y semánticas de ambos idiomas.

A partir de este ejemplo resuelve los siguientes ejercicios:

1 .  Implementa beam search en tu modelo de atención. Comienza con un factor de beam pequeño (por ejemplo, k=3) y experimenta con diferentes valores para ver cómo afecta la calidad de las predicciones.

In [ ]:
def beam_search_decoder(dec_outputs, k):
    sequences = [[list(), 1.0]]
    # Recorrer cada paso en la salida del decodificador
    for row in dec_outputs:
        all_candidates = list()
        # Expandir cada secuencia actual
        for seq, score in sequences:
            for j in range(len(row)):
                candidate = [seq + [j], score * -np.log(row[j])]
                all_candidates.append(candidate)
        # Ordenar todos los candidatos por puntuación
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        # Seleccionar k mejores
        sequences = ordered[:k]
    return sequences


2 . Modifica la decodificación para incorporar sampling con temperatura. La temperatura modifica la distribución de probabilidad de la salida del modelo, permitiendo controlar el grado de aleatoriedad en la elección de palabras.

In [ ]:
def temperature_sampling(logits, temperature=1.0):
    probs = F.softmax(logits / temperature, dim=-1)
    return torch.multinomial(probs, 1).item()


3 . Experimenta con diferentes tasas de dropout en las celdas RNN y añade regularización L2 al optimizador.

In [ ]:
# Modifica la tasa de dropout en las celdas RNN
self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)  # menor dropout
self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)

# Añadir regularización L2 al optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # añadir weight_decay para L2


4 . Varía el número de unidades ocultas, la tasa de aprendizaje, y el número de épocas de entrenamiento. Observa cómo cada cambio afecta el desempeño del modelo.

In [ ]:
## Tus respuestas

In [ ]:
# Referencia : https://github.com/prakashpandey9/Text-Classification-Pytorch/blob/master/models/LSTM_Attn.py
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, FixedFormatter

class BiLSTM_Attention(nn.Module):
    def __init__(self):
        super(BiLSTM_Attention, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, n_hidden, bidirectional=True)
        self.out = nn.Linear(n_hidden * 2, num_classes)

    # lstm_output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
    def attention_net(self, lstm_output, final_state):
        hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
        soft_attn_weights = F.softmax(attn_weights, 1)
        # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

    def forward(self, X):
        input = self.embedding(X) # input : [batch_size, len_seq, embedding_dim]
        input = input.permute(1, 0, 2) # input : [len_seq, batch_size, embedding_dim]

        hidden_state = torch.zeros(1*2, len(X), n_hidden) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        cell_state = torch.zeros(1*2, len(X), n_hidden) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        output, (final_hidden_state, final_cell_state) = self.lstm(input, (hidden_state, cell_state))
        output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
        attn_output, attention = self.attention_net(output, final_hidden_state)
        return self.out(attn_output), attention # model : [batch_size, num_classes], attention : [batch_size, n_step]

if __name__ == '__main__':
    embedding_dim = 2 # tamaño del embedding
    n_hidden = 5  # número de unidades ocultas en una celda
    num_classes = 2  # 0 o 1

    # Sentencias de 3 palabras (=la longitud de la secuencia es 3)
    sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
    labels = [1, 1, 1, 0, 0, 0]  # 1 es bueno, 0 no es bueno.

    word_list = " ".join(sentences).split()
    word_list = list(set(word_list))
    word_dict = {w: i for i, w in enumerate(word_list)}
    vocab_size = len(word_dict)

    model = BiLSTM_Attention()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    inputs = np.array([np.asarray([word_dict[n] for n in sen.split()]) for sen in sentences])
    inputs = torch.LongTensor(inputs)
    targets = torch.LongTensor([out for out in labels])  # Para usar la función de pérdida Softmax de Torch

    # Entrenamiento
    for epoch in range(5000):
        optimizer.zero_grad()
        output, attention = model(inputs)
        loss = criterion(output, targets)
        if (epoch + 1) % 1000 == 0:
            print('Epoca:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

    # Prueba
    test_text = 'sorry hate you'
    tests = np.array([np.asarray([word_dict[n] for n in test_text.split()])])
    test_batch = torch.LongTensor(tests)

    # Predicción
    predict, _ = model(test_batch)
    predict = predict.data.max(1, keepdim=True)[1]
    if predict[0][0] == 0:
        print(test_text,"is Bad Mean...")
    else:
        print(test_text,"is Good Mean!!")

    fig = plt.figure(figsize=(6, 3)) # [batch_size, n_step]
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    ax.set_xticks(range(len(['first_word', 'second_word', 'third_word'])))
    ax.set_xticklabels(['first_word', 'second_word', 'third_word'], fontdict={'fontsize': 14}, rotation=90)
    
    ax.set_yticks(range(len(['batch_1', 'batch_2', 'batch_3', 'batch_4', 'batch_5', 'batch_6'])))
    ax.set_yticklabels(['batch_1', 'batch_2', 'batch_3', 'batch_4', 'batch_5', 'batch_6'], fontdict={'fontsize': 14})
    
    plt.show()


El resultado representa una matriz de atención de un modelo BiLSTM con mecanismo de atención. En esta matriz:

* Las etiquetas en el eje X representan las palabras en las sentencias ("first_word", "second_word", "third_word").
* Las etiquetas en el eje Y representan los diferentes lotes ("batch_1", "batch_2", "batch_3", "batch_4", "batch_5", "batch_6").

Cada celda en la matriz muestra el peso de atención que el modelo asigna a cada palabra en las oraciones de entrada. Los colores indican la intensidad de estos pesos, donde los colores más claros (amarillo) representan mayores pesos de atención y los colores más oscuros (morado) representan menores pesos.

* Incrementar el tamaño de la capa de embedding y las unidades ocultas de LSTM para mejorar la capacidad del modelo de capturar características más complejas.
* Ajusta la tasa de aprendizaje y probar diferentes optimizadores como AdamW o RMSprop.
* Incluye más oraciones en el conjunto de datos para entrenar el modelo de manera más robusta.
* Mejora la visualización de la matriz de atención utilizando técnicas de matplotlib avanzadas para una interpretación más clara.
* Meustra las palabras reales en los ejes en lugar de "first_word", "second_word", etc.
* Agrega capas de Dropout después de la capa de LSTM para evitar el sobreajuste.
* Implementa técnicas de regularización como L2 regularization en los parámetros del modelo.

Presenta una versión mejorada del código que incorpora algunas de esas mejoras.

In [ ]:
## Tus respuestas

#### Mecanismo de atención jerárquica

**Descripción general**

La atención jerárquica se utiliza para manejar estructuras de datos complejas y de múltiples niveles, como documentos largos divididos en párrafos, párrafos divididos en oraciones y oraciones divididas en palabras. Este mecanismo aplica la atención en dos niveles: a nivel de palabra dentro de cada oración y a nivel de oración dentro del documento. Esta estructura permite capturar dependencias tanto locales como globales de manera eficiente.

**Ecuaciones y cálculos**

El proceso de atención jerárquica se puede dividir en dos fases principales:

1. **Atención a nivel de palabra**:
   Primero, se aplica la atención para cada palabra dentro de cada oración. Supongamos que una oración $ \text{sentence}_i $ contiene $ T_i $ palabras y sus representaciones de palabra son $ \{h_{i1}, h_{i2}, \ldots, h_{iT_i}\} $. La atención a nivel de palabra se calcula de la siguiente manera:

   $$
   e_{ij} = v_1^T \tanh(W_1 h_{ij} + b_1)
   $$

   donde $ W_1 $ y $ v_1 $ son parámetros aprendibles, y $ b_1 $ es un vector de sesgo.

   Los pesos de atención se obtienen aplicando la función softmax a los puntajes $ e_{ij} $:

   $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_i} \exp(e_{ik})}
   $$

   El vector de contexto para la oración $ i $ se calcula como una combinación ponderada de las representaciones de palabra:

   $$
   c_i = \sum_{j=1}^{T_i} \alpha_{ij} h_{ij}
   $$

2. **Atención a nivel de oración**:
   Una vez obtenidos los vectores de contexto $ \{c_1, c_2, \ldots, c_N\} $ para todas las oraciones en un documento (donde $ N $ es el número de oraciones en el documento), se aplica la atención a nivel de oración:

   $$
   e_i = v_2^T \tanh(W_2 c_i + b_2)
   $$

   Los pesos de atención a nivel de oración se obtienen aplicando la función softmax a los puntajes $ e_i $:

   $$
   \beta_i = \frac{\exp(e_i)}{\sum_{k=1}^{N} \exp(e_k)}
   $$

   El vector de contexto para el documento se calcula como una combinación ponderada de los vectores de contexto de las oraciones:

   $$
   d = \sum_{i=1}^{N} \beta_i c_i
   $$


La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención jerárquica:

In [ ]:
class HierarchicalAttention(nn.Module):
    def __init__(self, word_hidden_size, sentence_hidden_size, vocab_size, word_embedding_dim):
        super(HierarchicalAttention, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
        self.word_encoder = nn.GRU(word_embedding_dim, word_hidden_size, batch_first=True)
        self.word_attention = nn.Linear(word_hidden_size, word_hidden_size)
        self.sentence_encoder = nn.GRU(word_hidden_size, sentence_hidden_size, batch_first=True)
        self.sentence_attention = nn.Linear(sentence_hidden_size, sentence_hidden_size)
        self.fc = nn.Linear(sentence_hidden_size, num_classes)  # num_classes depende de la tarea específica

    def forward(self, documents):
        sentence_vectors = []
        for sentences in documents:  # Iterar sobre documentos
            word_vectors = []
            for sentence in sentences:  # Iterar sobre oraciones
                embedded_words = self.word_embedding(sentence)
                word_enc_outputs, _ = self.word_encoder(embedded_words)
                word_att_weights = F.softmax(self.word_attention(word_enc_outputs), dim=1)
                word_vector = torch.sum(word_att_weights * word_enc_outputs, dim=1)
                word_vectors.append(word_vector)
            sentence_vectors.append(torch.stack(word_vectors))
        sentence_enc_outputs, _ = self.sentence_encoder(torch.stack(sentence_vectors))
        sentence_att_weights = F.softmax(self.sentence_attention(sentence_enc_outputs), dim=1)
        doc_vector = torch.sum(sentence_att_weights * sentence_enc_outputs, dim=1)
        output = self.fc(doc_vector)
        return output


#### Mecanismo de atención basada en consultas

**Descripción general**

La atención basada en consultas, utilizada en modelos como el Transformer, utiliza tres componentes principales: consultas (queries), claves (keys) y valores (values). Este mecanismo permite calcular la atención como una función de similitud entre las consultas y las claves, aplicándola a los valores para obtener una representación ponderada. Este enfoque es altamente eficiente y escalable.

**Ecuaciones y cálculos**

El mecanismo de atención basada en consultas se define a través de los siguientes pasos:

1. **Cálculo de consultas, claves y valores**:
   Las consultas $ Q$, las claves $ K$ y los valores $ V$ se obtienen mediante proyecciones lineales de la entrada:

   $$
   Q = X W_Q, \quad K = X W_K, \quad V = X W_V
   $$

   donde $ W_Q$, $ W_K$ y $ W_V$ son matrices de peso aprendibles.

2. **Cálculo de puntajes de atención**:
   Los puntajes de atención se calculan como el producto punto escalado entre las consultas y las claves:

   $$
   e_{ij} = \frac{Q_i K_j^T}{\sqrt{d_k}}
   $$

   donde $ d_k$ es la dimensión de las claves.

3. **Normalización de puntajes de atención**:
   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención $ \alpha_{ij}$:

   $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k} \exp(e_{ik})}
   $$

4. **Cálculo del vector de contexto**:
   El vector de contexto $ c_i$ se calcula como una combinación ponderada de los valores:

   $$
   c_i = \sum_{j} \alpha_{ij} V_j
   $$


La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención basada en consultas

In [ ]:
class QueryBasedAttention(nn.Module):
    def __init__(self, hidden_size):
        super(QueryBasedAttention, self).__init__()
        self.linear_q = nn.Linear(hidden_size, hidden_size)
        self.linear_k = nn.Linear(hidden_size, hidden_size)
        self.linear_v = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, key, value):
        Q = self.linear_q(query)
        K = self.linear_k(key)
        V = self.linear_v(value)
        
        attn_scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_output = torch.matmul(attn_weights, V)
        
        output = self.fc_out(attn_output)
        return output

class Seq2SeqWithQueryBasedAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2SeqWithQueryBasedAttention, self).__init__()
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.GRU(output_dim, hidden_dim, batch_first=True)
        self.query_attn = QueryBasedAttention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, src, trg):
        enc_output, hidden = self.encoder(src)
        dec_output, _ = self.decoder(trg, hidden)
        attn_output = self.query_attn(dec_output, enc_output, enc_output)
        output = self.fc(attn_output)
        return output


Los mecanismos de atención jerárquica y basada en consultas ofrecen enfoques avanzados y efectivos para mejorar el rendimiento de los modelos de secuencia a secuencia en diversas tareas de procesamiento de lenguaje natural. La atención jerárquica es ideal para manejar datos estructurados jerárquicamente y capturar dependencias a múltiples niveles, mientras que la atención basada en consultas es altamente eficiente y escalable, lo que la hace fundamental para modelos modernos como el Transformer.


#### Mecanismo de auto-atención (Self-Attention)

**Descripción general**

El mecanismo de auto-atención, o self-attention, permite que cada elemento de la secuencia preste atención a todos los demás elementos de la misma secuencia. Esto es fundamental para capturar las dependencias a largo plazo en las secuencias y es un componente clave en los modelos Transformer.

**Ecuaciones y cálculos**

El proceso de auto-atención se puede describir mediante los siguientes pasos:

1. **Proyección lineal**:
   Al igual que en la atención multi-cabecera, se proyectan las consultas $Q$, las claves $K$ y los valores $V$:

   $$
   Q = X W_Q, \quad K = X W_K, \quad V = X W_V
   $$

   donde $W_Q$, $W_K$ y $W_V$ son matrices de peso aprendibles.

2. **Cálculo de puntajes de atención**:
   Los puntajes de atención se calculan utilizando el producto punto escalado:

   $$
   e_{ij} = \frac{Q_i K_j^T}{\sqrt{d_k}}
   $$

3. **Normalización de puntajes de atención**:
   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención  $\alpha_{ij}$:

   $$
   \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k} \exp(e_{ik})}
   $$

4. **Cálculo del vector de contexto**:
   El vector de contexto $c_i$ se calcula como una combinación ponderada de los valores:

   $$
   c_i = \sum_{j} \alpha_{ij} V_j
   $$


La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de auto-atención:

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size):
        super(SelfAttention, self).__init__()
        self.linear_q = nn.Linear(hidden_size, hidden_size)
        self.linear_k = nn.Linear(hidden_size, hidden_size)
        self.linear_v = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        Q = self.linear_q(x)
        K = self.linear_k(x)
        V = self.linear_v(x)

        attn_scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_output = torch.matmul(attn_weights, V)

        output = self.fc_out(attn_output)
        return output


#### Mecanismo de atención multi-cabecera (Multi-Head Attention)

**Descripción general**

El mecanismo de atención multi-cabecera, introducido por Vaswani en el modelo Transformer, extiende la idea de la auto-atención al permitir que el modelo se concentre en diferentes partes de la secuencia de entrada de manera simultánea y desde múltiples perspectivas. Esto se logra al tener múltiples "cabeceras" de atención, cada una de las cuales realiza una operación de atención independiente.

**Ecuaciones y cálculos**

El proceso de atención multi-cabecera se puede dividir en varios pasos:

1. **Proyección lineal**:
   Se proyectan las consultas  $Q$, las claves $K$ y los valores $V$ en subespacios diferentes para cada cabecera de atención. Supongamos que tenemos $h$ cabecera de atención y una dimensión de modelo $d_{\text{model}}$. La proyección se realiza de la siguiente manera:

   $$
   Q_h = X W_Q^h, \quad K_h = X W_K^h, \quad V_h = X W_V^h
   $$

   donde $W_Q^h$, $W_K^h$ y $W_V^h$ son matrices de peso específicas para la cabecera  $h$.

2. **Cálculo de puntajes de atención**:
   Para cada cabecera de atención, se calcula el puntaje de atención utilizando el producto punto escalado:

   $$
   e_{ij}^h = \frac{Q_i^h (K_j^h)^T}{\sqrt{d_k}}
   $$

   donde $d_k$ es la dimensión de las claves.

3. **Normalización de puntajes de atención**:

   Los puntajes de atención se normalizan usando la función softmax para obtener los pesos de atención $\alpha_{ij}^h$:

   $$
   \alpha_{ij}^h = \frac{\exp(e_{ij}^h)}{\sum_{k} \exp(e_{ik}^h)}
   $$

4. **Cálculo del vector de contexto**:
   El vector de contexto para cada cabecera de atención se calcula como una combinación ponderada de los valores:

   $$
   c_i^h = \sum_{j} \alpha_{ij}^h V_j^h
   $$

5. **Concatenación y proyección final**:
   Los vectores de contexto de todas las cabeceras se concatenan y se proyectan de nuevo en el espacio original:

   $$
   \text{MultiHead}(Q, K, V) = \text{Concat}(c_i^1, c_i^2, \ldots, c_i^h) W_O
   $$

   donde  $W_O$ es la matriz de peso de proyección final.

La siguiente implementación en PyTorch muestra cómo se puede construir un mecanismo de atención multi-cabecera:

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert hidden_size % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        self.linear_q = nn.Linear(hidden_size, hidden_size)
        self.linear_k = nn.Linear(hidden_size, hidden_size)
        self.linear_v = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, key, value):
        batch_size = query.size(0)

        # Proyección lineal
        Q = self.linear_q(query).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.linear_k(key).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.linear_v(value).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        # Cálculo de puntajes de atención
        attn_scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.head_dim)

        # Normalización de puntajes de atención
        attn_weights = F.softmax(attn_scores, dim=-1)

        # Cálculo del vector de contexto
        attn_output = torch.matmul(attn_weights, V)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.head_dim)

        # Proyección final
        output = self.fc_out(attn_output)
        return output


Estos mecanismos son fundamentales para el funcionamiento de los modelos Transformer, permitiendo capturar dependencias a largo plazo y manejar grandes cantidades de datos de manera eficiente. 

### Ejercicios

1 .Implementa un modelo de atención jerárquica donde primero se aplique atención a nivel de palabra y luego a nivel de frase para tareas de resumen de texto o clasificación de documentos.

Instrucciones:

- Divide un documento en oraciones y cada oración en palabras.
- Implementa atención a nivel de palabra para obtener una representación de cada oración.
- Implementa atención a nivel de frase para obtener una representación del documento.
- Usa la representación del documento para realizar una tarea específica (por ejemplo, clasificación de documentos).

In [ ]:
## Tu respuesta

2 . Implementa un mecanismo de atención multi-cabecera similar al utilizado en los modelos de transformadores, para permitir que el modelo enfoque en diferentes partes de la entrada de manera simultánea.

Instrucciones:

- Implementa un bloque de atención con múltiples cabezas.
- Integra este bloque en un modelo seq2seq.
- Evalua el rendimiento del modelo en una tarea de traducción automática.

In [ ]:
## Tu respuesta

3 . Implementa un modelo de atención local que solo considere una ventana alrededor de la posición actual en lugar de toda la secuencia de entrada, reduciendo la complejidad computacional.

Instrucciones:

* Define una ventana de atención fija.
* Implementa el cálculo de los pesos de atención solo dentro de esta ventana.
* Integra este mecanismo en un modelo seq2seq y evaluar su rendimiento

In [ ]:
## Tu respuesta

4 . Implementa un modelo de atención basada en consultas similar al mecanismo utilizado en el Transformador, donde las consultas, las claves y los valores provienen de proyecciones de la entrada.

Instrucciones:

- Implementa las proyecciones lineales para consultas, claves y valores.
- Calcula los pesos de atención usando productos escalares entre las consultas y las claves.
- Aplica estos pesos a los valores para obtener la representación de atención

In [ ]:
## Tu respuesta

5 . Implementa la auto-atención donde cada elemento de la secuencia presta atención a todos los demás elementos de la misma secuencia. Esto es útil para tareas de traducción automática y clasificación de secuencias.

Instrucciones:

- Implementa el cálculo de auto-atención utilizando proyecciones lineales para claves, consultas y valores.
- Integra el mecanismo de auto-atención en un modelo seq2seq.
- Evalua el rendimiento del modelo en una tarea de traducción automática.

In [ ]:
## Tu respuesta